In [1]:
import torch.nn as  nn

In [2]:
class aggregation_coder(nn.Module):
    def __init__(self,in_dim,hidden_dim,out_dim):
        super(aggregation_coder,self).__init__()
        self.l1=nn.Linear(in_dim,hidden_dim)
        self.l2=nn.Linear(hidden_dim,out_dim)
        self.l3=nn.Linear(hidden_dim,out_dim)
        self.l4=nn.Linear(hidden_dim,out_dim)
        nn.init.xavier_normal_(self.l1.weight,gain=1.414)
        nn.init.xavier_normal_(self.l2.weight,gain=1.414)
    def forward(self,shuru):
        out=self.l1(shuru)
        out=F.leaky_relu(out)
    #    out=F.leaky_relu(self.l2(out))
     #   out=F.leaky_relu(self.l3(out))
     #   out=F.leaky_relu(self.l4(out))
     #   out=self.l2(out)
        return out

In [3]:
class MAGNN_ctr_ntype_specific(nn.Module):
    def __init__(self,
                 num_metapaths,
                 etypes_list,
                 out_dim,
                 num_heads,
                 attn_vec_dim,
                 attn_drop=0.5,
                 use_minibatch=False):
        super(MAGNN_ctr_ntype_specific, self).__init__()
        self.out_dim = out_dim
        self.num_heads = num_heads
        self.use_minibatch = use_minibatch
        self.emb=[]
        # metapath-specific layers
        self.metapath_layers = nn.ModuleList()
        self.aggregation_coder=aggregation_coder(out_dim*3,out_dim,out_dim)
        for i in range(num_metapaths):
            self.metapath_layers.append(MAGNN_metapath_specific(etypes_list[i],
                                                                out_dim,
                                                                num_heads,
                                                                attn_drop=attn_drop,
                                                                use_minibatch=use_minibatch))

        # metapath-level attention
        # note that the acutal input dimension should consider the number of heads
        # as multiple head outputs are concatenated together
        self.fc1 = nn.Linear(out_dim * num_heads, attn_vec_dim, bias=True)
        self.fc2 = nn.Linear(attn_vec_dim, 1, bias=False)
        self.drop10=nn.Linear(out_dim*num_heads,attn_vec_dim,bias=True)
        self.drop11=nn.Linear(attn_vec_dim, 1, bias=False)
        self.drop20=nn.Linear(out_dim*num_heads,attn_vec_dim,bias=True)
        self.drop21=nn.Linear(attn_vec_dim, 1, bias=False)
        self.drop30=nn.Linear(out_dim*num_heads,attn_vec_dim,bias=True)
        self.drop31=nn.Linear(attn_vec_dim, 1, bias=False)
        
        
        # weight initialization
        nn.init.xavier_normal_(self.fc1.weight, gain=1.414)
        nn.init.xavier_normal_(self.fc2.weight, gain=1.414)
        nn.init.xavier_normal_(self.drop10.weight, gain=1.414)
        nn.init.xavier_normal_(self.drop11.weight, gain=1.414)
        nn.init.xavier_normal_(self.drop20.weight, gain=1.414)
        nn.init.xavier_normal_(self.drop21.weight, gain=1.414)
        nn.init.xavier_normal_(self.drop30.weight, gain=1.414)
        nn.init.xavier_normal_(self.drop31.weight, gain=1.414)
   
    def forward(self, inputs):
   
        if self.use_minibatch:
            g_list, features, type_mask, edge_metapath_indices_list, target_idx_list,node_list,num_list = inputs
            # metapath-specific layers
            metapath_outs = [F.elu(metapath_layer((g, features, type_mask, edge_metapath_indices, target_idx,nodes,num)).view(-1, self.num_heads * self.out_dim))
                             for g, edge_metapath_indices, target_idx, nodes,num,metapath_layer in zip(g_list, edge_metapath_indices_list, target_idx_list,node_list,num_list,self.metapath_layers)]
        else:
            g_list, features, type_mask, edge_metapath_indices_list,node_list,num_list = inputs

            # metapath-specific layers
            metapath_outs = [F.elu(metapath_layer((g, features, type_mask, edge_metapath_indices)).view(-1, self.num_heads * self.out_dim))
                             for g, edge_metapath_indices, metapath_layer in zip(g_list, edge_metapath_indices_list, self.metapath_layers)]
        ceta=torch.cat(metapath_outs,dim=1)
        h=self.aggregation_coder(ceta)
        return h